In [3]:
%%file hilmail.py
from mrjob.job import MRJob
from mrjob.step import MRStep

# map-reduce пример для работы с данным
# найдем топ-3 контакта из почты данной персоны (Hillary Clinton)

class MRTopSenders(MRJob):

    def mapper(self, key, record):
        # маппер
        # на вход подаются "грязные данные", не все данные подходят в CSV формат
        # пропустим первую строчку 
        # нужные данные начинаются с 20 строки (используем провеку >20)
        splits = record.split(",")
        if len(splits) > 20:
            # пропустим строки, где "to" или "from" пустые
            if len(splits[3]) and len(splits[4]):
                # получаем набор из кортежей (key (связка to -> from) и value)
                yield (splits[3],splits[4]), 1
                

    def sum_by_from(self, tofrom, count):
        # сделаем связку "to", (count,"from")
        # этот шаг позволит далее отсортировать отправителей по кол-во
        yield tofrom[0], (sum(count),tofrom[1])
        

    def find_top_three(self, to, counts):
        topsenders = sorted(counts)[0:3]
        fromnames = [ ts[1] for ts in topsenders ]
        # результат "to","from1,from2,from3"
        yield to, ",".join(fromnames)
        

    def hillary_only(self,to,topsenders):
        # посмотрим, только Хиллари
        if to == "H":
        yield to, topsenders


    def steps(self):
        return [ MRStep(mapper=self.mapper, reducer=self.sum_by_from),
                 MRStep(reducer=self.find_top_three),
                 MRStep(mapper=self.hillary_only) ]


if __name__ == '__main__':
    MRTopSenders.run()

Overwriting email_connections.py


**Дайте ответ на вопрос:**

    - как можно сделать анализ социальный связий и анализ глубины связи по данному материалу (базируйтесь на to -> from)